In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
from alphabase.spectral_library import base
from alphadia import outputtransform
from alphadia.workflow import reporting

/Users/georgwallmann/miniconda3/envs/alpha/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
base_spec_lib = base.SpecLibBase()
base_spec_lib.load_hdf('/Users/georgwallmann/Documents/data/alphadia_manuscript/experiments/2023_12_13_mixed_species_astral/second_pass_polon_fragcomp/speclib.hdf', load_mod_seq=True)

progress_folder = '/Users/georgwallmann/Documents/data/alphadia_manuscript/experiments/2023_12_13_mixed_species_astral/second_pass_polon_fragcomp/.progress'
folder_list = [os.path.join(progress_folder, f) for f in os.listdir(progress_folder) if os.path.isdir(os.path.join(progress_folder, f))]


output_folder = '/Users/georgwallmann/Documents/data/alphadia_manuscript/experiments/2023_12_13_mixed_species_astral/second_pass_polon_fragcomp_k=12'


In [ ]:
config = {
        "general": {
            "thread_count": 8,
        },
        "fdr": {
            "fdr": 0.01,
            "inference_strategy": 'maximum_parsimony',
            "group_level": "genes",
            "keep_decoys": False,
        },
        "search_output": {
            "min_k_fragments": 12,
            "min_correlation": 0.9,
            "num_samples_quadratic": 50,
            "min_nonnan": 3,
            "peptide_level_lfq": False,
            "precursor_level_lfq": False,
            "normalize_lfq": True,
        },
    }

output = outputtransform.SearchPlanOutput(config, output_folder)
psm_df = output.build(folder_list, base_spec_lib)